# Two State and One Control Bellman Equation Model


**Suraj Kumar**  
**Suraj Kumar** 

**Post Graduation**  
*MA Economics(Final Year)*  
*Department of Economics*  
**Delhi School of Economics**  
*India*

**Under graduation**    
*B.Tech in Civil Engineering from **Indian Institute of Technology, Delhi (IIT Delhi)***

surajdhunna@gmail.com     
https://github.com/iamsurajkumar/Dynamic_Programming_Squared_Model

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from numpy import random
from scipy.stats import lognorm
from scipy.integrate import fixed_quad
from scipy import interpolate
from scipy import optimize

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from numba import jit


init_notebook_mode(connected = True)

class LucasAssetModel:
    """
    Lucas Asset Price Tree model
    
    see the above equations
    
    This class all model parameters 
    """
    
    def __init__(self,
                γ = 2,
                β = 0.95,
                α = 0.9,
                μ = 0,
                s = 0.1,
                rent_min = 1e-5,
                rent_max = 4,
                rent_grid_size = 100,
                shock_size = 100,
                seed = 100,
                π_min = 1e-5,
                π_max = 1,
                π_grid_size = 100):
        """
        Creates Instance of Lucas Model
        
        """
        ### Assinging Parameters
        
        # Model Parameters
        self.γ, self.β, self.α = γ, β, α
        
        # Shocks Parameters
        self.s, self.μ, self.shock_size, self.seed  = s, μ, shock_size, seed
        
        #Rental Grid State Allocation
        self.rent_min, self.rent_max, self.rent_grid_size  = rent_min, rent_max, rent_grid_size
        self.rent_grid = np.linspace( self.rent_min, self.rent_max, self.rent_grid_size)
        
        #Setting up shocks in the models
        np.random.seed(self.seed) # setting seed to generate same random numbers
        self.shocks = np.exp( self.μ + self.s * np.random.randn(self.shock_size)  )
        
        #Setting up grid for share π
        self.π_min, self.π_max, self.π_grid_size = π_min, π_max, π_grid_size
        self.π_grid = np.linspace(self.π_min, self.π_max, self.π_grid_size)

#@jit(nopython = True)
def u(π_prime, π, y, p, γ):
    cons = (π*(y + p)) - (π_prime*p)
    if ( (cons > 0) & (γ != 1) ):
        #return ( cons**(1-γ) )/(1-γ)
        return np.sqrt(cons)
    else:
        return -10e6

#@jit(nopython = True)
def g(y,z,α):
    return (y**α)*z

#@jit(nopython = True)
def p(y):
    return y**2

#@jit(nopython = True)
def initial_w(lam): #Intialzing the grid with some continous, concave function
    pi_s,rent_s = np.meshgrid(lam.π_grid, lam.rent_grid)
    return 5*np.sqrt(pi_s*rent_s)


def initial_w2(lam): #Intialzing the grid with some continous, concave function
    pi_s,rent_s = np.meshgrid(lam.π_grid, lam.rent_grid)
    return np.power(pi_s*rent_s,1/3)
    

# Writing my own Minimum Function
def min_fun(objective,π_grid, ans = None):
    if ans is None:
        ans = np.empty_like(π_grid)
    for i, π in enumerate(π_grid):
        ans[i] = objective(π)
    return π_grid[ans == min(ans)]



def bellman_operator(lam,w,u,g,p,Tw = None, compute_policy = 0):
    """
    Parameters
    ------------
    lam = instance of Lucas Asset Model
          stores the paramters of the problem    
    w = is mesh of different pi values and rent values
    u = utility function
    f = rental productin function
    p = price as function of rents y
    Tw = array to output values, Tw is just for space optimization
    compute_policy = to decide whether to ouput policy function or not

    """
    
    # Parameters Extractions
    α, β, γ = lam.α, lam.β, lam.γ 
    
    #Share Grid Extraction
    π_max, π_min,π_grid = lam.π_max, lam.π_min, lam.π_grid
    
    #Rent Grid Extraction
    rent_grid = lam.rent_grid
    
    #Shocks Grid Extract
    shocks = lam.shocks
    
    # === Appyling Linear interpolation to a two dimensional surface === #
    interpolation_func = interpolate.interp2d(π_grid, rent_grid, w, kind='linear')
    
   
    # === Intitiate Tw if required === #
    if Tw is None:
        Tw = np.empty_like(w)

    # === Intitate compute policy if asked === #

    if compute_policy:
        π_opt_policy = np.empty_like(w)

    for i, y in enumerate(rent_grid):
        for j, π in enumerate(π_grid):
            def objective(π_prime):
                return - u(π_prime, π, y, p(y), γ) - β * np.mean( interpolation_func( π_prime,g(y,shocks,α) ) )
            π_star = optimize.fminbound(objective, π_min, π_max)  
            #π_star = min_fun(objective,π_grid)
            
            if compute_policy:
                π_opt_policy[i,j] = π_star
            Tw[i,j] = - objective(π_star)

    if compute_policy:
        return Tw, π_opt_policy
    else:
        return Tw


def solve_optvalue(lam = LucasAssetModel(),
                   initial_func = initial_w,
                   tol = 1e-6, max_iter = 100):
    """
    This function is used to solve for 
    
    lam = creating 
    
    """
    w = initial_func(lam)
    d={}
    error = tol + 1
    i = 0
    d[0] = w
    
    #Create Storage for bellman Operator
    Tw = np.empty_like(w)
    
    #Iterate to find Solution
    while error > tol and i < max_iter:
        w_new = bellman_operator(lam,w,u,g,p,Tw = None, compute_policy = 0)
        error = np.max(np.abs(w_new - w))
        w = w_new
        i += 1
        d[i] = w_new    
    
    return w,i,d

In [4]:
mylam =LucasAssetModel(rent_grid_size=3,shock_size=10,π_grid_size=3)
w, i, d = solve_optvalue(lam= mylam,initial_func= initial_w,max_iter = 100 )

In [27]:
n = 200
lam =LucasAssetModel(rent_min=0.5,π_min=0.1,rent_grid_size=10,shock_size=10,π_grid_size=10)
w = initial_w(lam)
d={}
d[0] = w
#d[0] = go.Surface(z=w,colorscale='Viridis')

for i in range(n):
    w = bellman_operator(lam,w,u,g,p,Tw = None, compute_policy = 0)
    #fig = plt.figure()
    #ax = plt.axes(projection='3d')
    #ax.plot_surface(X, Y, w, rstride=1, cstride=1,cmap='viridis', edgecolor='none')
    #ax.set_title('surface {}'.format(i));
    #ax.view_init(60, 35)
    #fig
    #d[i+1] = go.Surface(z=w,showscale = True,colorscale='Viridis')
    d[i+1] = w

#py.iplot([trace,trace2])


error = np.abs(d[200] - d[199])
error.shape

fig = plt.figure()
#ax = plt.axes(projection='3d')
#ax.contour3D(X, Y, w, 50, cmap='viridis')
#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_zlabel('z')
#ax.view_init(60, 35)
#fig
X,Y = np.meshgrid(lam.π_grid,lam.rent_grid)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, d[50], rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
ax.set_title('surface');
#ax.view_init(60, 35)
#fig

In [73]:
mylam =LucasAssetModel(rent_min=0.05,π_min=0.01,rent_grid_size=10,shock_size=10,π_grid_size=10)
w2, i2, d2 = solve_optvalue(lam= mylam,initial_func= initial_w2,max_iter = 300 )
i2

268

4.8101322654263114